In [0]:
#to read files from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -rf data

In [0]:
!git clone https://github.com/udacity/dermatologist-ai.git
!mkdir data

!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip
    
!unzip test.zip -d data
!unzip train.zip -d data
!unzip valid.zip -d data

Cloning into 'dermatologist-ai'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.
--2019-05-24 08:20:46--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.24.17
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.24.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5736557430 (5.3G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   5.34G  69.3MB/s    in 73s     

2019-05-24 08:21:59 (75.0 MB/s) - ‘train.zip’ saved [5736557430/5736557430]

--2019-05-24 08:22:01--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.1
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.21

In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import transforms, datasets, models
import numpy as np
from collections import OrderedDict
import os

In [0]:
# keeping only 30% files nevus files in train and valid directory
dir_train = 'data/train/nevus/'
dir_valid = 'data/valid/nevus/'

files_train = os.listdir(dir_train)
files_valid = os.listdir(dir_valid)
print(len(files_train), len(files_train))

1372 1372


In [0]:
# keeping only 30% files nevus files in train and valid directory
dir_train = 'data/train/melanoma/'
dir_valid = 'data/valid/melanoma/'

files_train = os.listdir(dir_train)
files_valid = os.listdir(dir_valid)
print(len(files_train), len(files_train))

374 374


In [0]:
# keeping only 30% files nevus files in train and valid directory
dir_train = 'data/train/seborrheic_keratosis/'
dir_valid = 'data/valid/seborrheic_keratosis/'

files_train = os.listdir(dir_train)
files_valid = os.listdir(dir_valid)
print(len(files_train), len(files_train))

254 254


In [0]:
for file in files_train[int(0.3*len(files_train)):]:
  os.remove(dir_train+file)
  
for file in files_valid[int(0.3*len(files_train)):]:
  os.remove(dir_valid+file)

files_train = os.listdir(dir_train)
files_valid = os.listdir(dir_valid)

print(len(files_train), len(files_train))

76 76


In [0]:
train_on_gpu = torch.cuda.is_available()

data_dir = 'data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

batch_size = 64

train_transforms = transforms.Compose([transforms.Resize(300),
                                transforms.RandomRotation(5),
                                transforms.RandomCrop(224),
                                transforms.RandomHorizontalFlip(0.1),
                                transforms.RandomVerticalFlip(0.1),
                                transforms.ToTensor()])

valid_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)

In [0]:
valid_data.class_to_idx

{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}

In [0]:
model = models.resnet152(pretrained=True)
#model

In [0]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.xavier_normal_(m.weight)
    
ct = 0
for child in model.children():
  ct += 1
  if ct < 4:
    for param in child.parameters():
       param.requires_grad = False

classifier_name, old_classifier = model._modules.popitem()

#add new hidden and dropout layers at the end
classifier_input_size = old_classifier.in_features
print(classifier_input_size)
classifier = nn.Sequential(OrderedDict([('drop1', nn.Dropout(p=0.45)),
                                           ('leakyrelu1', nn.LeakyReLU(0.08)),
                                           ('drop2', nn.Dropout(p=0.4)),
                                           ('fc1', nn.Linear(classifier_input_size, 512)),
                                           ('batchnorm1', nn.BatchNorm1d(512)),
                                           ('leakyrelu2', nn.LeakyReLU(0.004)),
                                           ('drop3', nn.Dropout(p=0.5)),
                                             ('fc2', nn.Linear(512, 3))
                                           ]))

classifier.apply(init_normal)
model.add_module(classifier_name, classifier)

2048


In [0]:
#defining hyper-parameters
epochs = 25
learning_rate = 0.005

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.85)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2)

def get_lr(optimizer):
  for param_group in optimizer.param_groups:
      return param_group['lr']

In [0]:
# helper scale function
def scale(x, feature_range=(-1, 1)):
    ''' Scale takes in an image x and returns that image, scaled
       with a feature_range of pixel values from -1 to 1. 
       This function assumes that the input x is already scaled from 0-1.'''
    Y = feature_range[0]
    X = feature_range[1] - feature_range[0]
    x = x * X + Y
    
    return x

In [0]:
#training the model
if train_on_gpu:
    model.cuda()

valid_loss_min = np.Inf

for epoch in range(epochs):
  model.train()
  # keep track of training and validation loss
  train_loss = 0.0
  valid_loss=0.0
  accuracy = 0.0
  running_corrects = 0.0
  dataset_size = 0
    
  for images, labels in trainloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
      images, labels = images.cuda(), labels.cuda()
    #images = scale(images)
    # clear the gradients of all optimized variables
    optimizer.zero_grad()

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(images)

    # calculate the batch loss
    loss = criterion(output, labels)

    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()

    # perform a single optimization step (parameter update)
    optimizer.step()

    # update training loss
    train_loss += loss.item()
        
  #validation starts here
  model.eval()
  with torch.no_grad():
    for images, labels in validloader:
      if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
      #images = scale(images)
      output = model(images)
      loss = criterion(output, labels)
      valid_loss += loss.item()

      _, preds = torch.max(output, 1)
      running_corrects += torch.sum(preds == labels.data)
      dataset_size += len(images)
        
  lr_scheduler.step(valid_loss_min)
  # Calculate accuracy
  accuracy = running_corrects.double() / dataset_size
  #print('learning rate is :', get_lr(optimizer))

  print("Epoch #", epoch, ':\t', "Training loss: ({:.3f})".format(train_loss/len(trainloader)), '\t',
        "Validation loss: ({:.3f})".format(valid_loss/len(validloader)), '\t',
        "Validation accuracy: ({:.3f})".format(accuracy))
    
  # save model if validation loss has decreased
  if valid_loss/len(validloader) <= valid_loss_min:
    print('Validation loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(
    valid_loss_min,
    valid_loss/len(validloader)))
    torch.save(model.state_dict(), 'model_dermatologist_ai.pt')
    valid_loss_min = valid_loss/len(validloader)

Epoch # 0 :	 Training loss: (1.171) 	 Validation loss: (1.020) 	 Validation accuracy: (0.547)
Validation loss decreased (inf --> 1.020).  Saving model ...
Epoch # 1 :	 Training loss: (0.989) 	 Validation loss: (0.834) 	 Validation accuracy: (0.653)
Validation loss decreased (1.020 --> 0.834).  Saving model ...
Epoch # 2 :	 Training loss: (0.919) 	 Validation loss: (0.731) 	 Validation accuracy: (0.660)
Validation loss decreased (0.834 --> 0.731).  Saving model ...
Epoch # 3 :	 Training loss: (0.770) 	 Validation loss: (0.671) 	 Validation accuracy: (0.620)
Validation loss decreased (0.731 --> 0.671).  Saving model ...
Epoch # 4 :	 Training loss: (0.694) 	 Validation loss: (0.686) 	 Validation accuracy: (0.713)
Epoch # 5 :	 Training loss: (0.594) 	 Validation loss: (0.662) 	 Validation accuracy: (0.687)
Validation loss decreased (0.671 --> 0.662).  Saving model ...
Epoch # 6 :	 Training loss: (0.513) 	 Validation loss: (0.766) 	 Validation accuracy: (0.713)
Epoch # 7 :	 Training loss: (

In [0]:
#upload the checkpoint to the drive
!cp model_dermatologist_ai.pt './drive/My Drive/Pytorch_phase2/dermatologist_ai/model_dermatologist_ai_84.pt'

cp: cannot stat 'model_dermatologist_ai.pt': No such file or directory


In [0]:
#loading the best most accurate checkpoint
state_dict = torch.load('./drive/My Drive/Pytorch_phase2/dermatologist_ai/Copy of model_dermatologist_ai_84.pt')
model.load_state_dict(state_dict)
#state_dict
#model

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

test_data = ImageFolderWithPaths(test_dir, transform=valid_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [0]:
import pandas as pd
#classes = [name.replace('_', ' ') for name in test_data.classes]
#print(classes)
if train_on_gpu:
    model.cuda()
#creating dictionary and lists for the csv file
csv_dict = {}
id_values = []
task_1_values = []
task_2_values = []

#testing the model
model.eval()
test_loss = 0.0
dataset_size = 0
running_corrects = 0

with torch.no_grad():
  for images, labels, paths in testloader:
    if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
    #images = scale(images)
    output = model(images)
    loss = criterion(output, labels)
    test_loss += loss.item()

    _, preds = torch.max(output, 1)
    running_corrects += torch.sum(preds == labels.data)
    dataset_size += len(images)
    
    output = nn.functional.softmax(output, dim=1)
    #print(output[0:5])
    
    #adding values to the columns
    id_values.extend(paths)
    #print(id_values)
    
    for each_output in output:
      task_1_values.append(each_output.tolist()[0])
      task_2_values.append(each_output.tolist()[2])
      
    #print(task_1_values)
    #print(task_2_values)

# Calculate accuracy
accuracy = running_corrects.double() / dataset_size

print("Test loss: ({:.3f})".format(test_loss/len(testloader)), '\t',
      "Test accuracy: ({:.3f})".format(accuracy))

Test loss: (1.829) 	 Test accuracy: (0.617)


In [0]:
#exporting predictions to a csv file
csv_dict['Id'] = id_values
csv_dict['task_1'] = task_1_values
csv_dict['task_2'] = task_2_values

df = pd.DataFrame.from_dict(csv_dict, orient = 'columns')
df.to_csv('predictions_813.csv', index = False)

In [0]:
#upoading the csv file to gdrive
!cp predictions_813.csv './drive/My Drive/dermatologist_ai/predictions_813.csv'

In [0]:
#installing requirements to run evaluations
!pip install -r dermatologist-ai/requirements.txt

In [0]:
# change the working directory
os.chdir('./dermatologist-ai')

In [0]:
#evaluating the predictions
!python './get_results.py' '/content/drive/My Drive/dermatologist_ai/predictions_84.csv' 0.8